# Example of upper limit computation.

In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
import zfit
from utils import plotfitresult, plotlimit, pltdist
from zfit.loss import ExtendedUnbinnedNLL
from zfit.minimize import Minuit

from hepstats.hypotests import UpperLimit
from hepstats.hypotests.calculators import FrequentistCalculator
from hepstats.hypotests.parameters import POI, POIarray

In [ ]:
plt.rcParams["figure.figsize"] = (9, 6)
plt.rcParams["font.size"] = 16

### Fit of a Gaussian signal over an exponential background:

In [ ]:
bounds = (0.1, 3.0)

# Data and signal

np.random.seed(0)
tau = -2.0
beta = -1 / tau
data = np.random.exponential(beta, 300)
peak = np.random.normal(1.2, 0.1, 10)
data = np.concatenate((data, peak))
data = data[(data > bounds[0]) & (data < bounds[1])]

In [ ]:
pltdist(data, bins=80, bounds=bounds)

In [ ]:
obs = zfit.Space("x", limits=bounds)

In [ ]:
lambda_ = zfit.Parameter("lambda", -2.0, -4.0, -1.0)
Nsig = zfit.Parameter("Nsig", 1.0, -20.0, len(data))
Nbkg = zfit.Parameter("Nbkg", len(data), 0.0, len(data) * 1.1)

In [ ]:
signal = zfit.pdf.Gauss(obs=obs, mu=1.2, sigma=0.1).create_extended(Nsig)
background = zfit.pdf.Exponential(obs=obs, lambda_=lambda_).create_extended(Nbkg)
tot_model = zfit.pdf.SumPDF([signal, background])

In [ ]:
# Create the negative log likelihood
data_ = zfit.data.Data.from_numpy(obs=obs, array=data)
nll = ExtendedUnbinnedNLL(model=tot_model, data=data_)

In [ ]:
# Instantiate a minuit minimizer
minimizer = Minuit()

In [ ]:
# minimisation of the loss function
minimum = minimizer.minimize(loss=nll)
minimum.hesse()

In [ ]:
nbins = 80
pltdist(data, nbins, bounds)
plotfitresult(tot_model, bounds, nbins)
plt.xlabel("m [GeV/c$^2$]")
plt.ylabel("number of events")

### Upper limit:

Since the signal yield Nsig = $4.5 \pm 5.8$  is consistent with zero signal we compute the CLs upper limit at 95% confidence level on Nsig.

In [ ]:
# instantation of the calculator
# calculator = FrequentistCalculator(nll, minimizer, ntoysnull=5000, ntoysalt=5000)
calculator = FrequentistCalculator.from_yaml(
    "toys/upperlimit_freq_zfit_toys.yml", nll, minimizer, ntoysnull=5000, ntoysalt=5000
)
calculator.bestfit = minimum  # optionnal

In [ ]:
# parameter of interest of the null hypothesis
poinull = POIarray(Nsig, np.linspace(0.0, 25, 15))
# parameter of interest of the alternative hypothesis
poialt = POI(Nsig, 0.0)

In [ ]:
# instantation of the discovery test
ul = UpperLimit(calculator, poinull, poialt)

In [ ]:
ul.upperlimit(alpha=0.05, CLs=True);

In [ ]:
f = plt.figure(figsize=(9, 8))
plotlimit(ul, alpha=0.05, CLs=True)
plt.xlabel("Nsig");

In [ ]:
f = plt.figure(figsize=(9, 8))
ax = plotlimit(ul, alpha=0.05, CLs=False)
plt.xlabel("Nsig");

In [ ]:
calculator.to_yaml("toys/upperlimit_freq_zfit_toys.yml")

### Illustration of the computation of the pvalues:

In the following we plot the q, the profile likelihood ratio, distributions for the null (red) and alternative hypothesis (blue) as well as the observed value q_obs.

In [ ]:
qnull = calculator.qnull(poinull, poialt, onesided=True)

In [ ]:
qobs = calculator.qobs(poinull, onesided=True)

In [ ]:
qalt = calculator.qalt(poinull, poialt, onesided=True)

In [ ]:
nbins = 30
fontsize = 20


def compute_pvalue(qdist, qobs):
    qdist = qdist[~(np.isnan(qdist) | np.isinf(qdist))]
    return len(qdist[qdist >= qobs]) / len(qdist)


f, axs = plt.subplots(5, 3, figsize=(25, 30), tight_layout=True)

for i, p in enumerate(poinull):
    ax = axs.flatten()[i]

    qaltp = qalt[p]
    qaltp = qaltp[~np.isnan(qaltp)]

    bounds = (0, np.percentile(qaltp, 99.9))

    ax.hist(
        qnull[p],
        bins=nbins,
        label="qnull distribution",
        log=True,
        range=bounds,
        histtype="step",
        color="crimson",
        lw=2,
    )
    ax.hist(
        qaltp,
        bins=nbins,
        label="qalt distribution",
        log=True,
        range=bounds,
        histtype="step",
        color="royalblue",
        lw=2,
    )
    ax.axvline(qobs[i], color="black", label="qobs", lw=2)
    ax.set_title(f"{p.name} = {p.value:.2f}")
    ax.legend(loc="best", fontsize=fontsize)
    ax.set_xlabel("q", fontsize=fontsize)

    p_clsb = compute_pvalue(qnull[p], qobs[i])
    p_clb = compute_pvalue(qaltp, qobs[i])
    p_cls = p_clsb / p_clb

    textstr = r"$p_{CL_{B}}$" + f"     = {p_clb:.2f}\n"
    textstr += r"$p_{CL_{S+B}}$" + f"  = {p_clsb:.2f}\n"
    textstr += r"$p_{CL_{S}}$" + f"     = {p_cls:.2f}"

    props = {"boxstyle": "round", "facecolor": "white", "alpha": 0.5}
    ax.text(
        0.65,
        0.65,
        textstr,
        transform=ax.transAxes,
        fontsize=fontsize,
        verticalalignment="top",
        bbox=props,
    )